In [ ]:
from collections import namedtuple
from pipeline import Pipeline
margs = namedtuple('args', 'env arena_config num_episodes')
env_path = '../env/macro2'
ac = "configurations/curriculum4/25.yml"
# ac = '../competition_configurations/5-10-1.yml'
args = margs(env=env_path, arena_config=ac, num_episodes=20)
pip = Pipeline(args)

In [ ]:
x = pip.run()

In [ ]:
df = x['t']['objects']
df1 = df[ 'visible']
df1 = df1[df1!=-1].index.tolist()
visible = ""
for i in df1:
    visible += f"visible({round(i*-1)})\n"
visible

In [ ]:
from clyngor import ASP

lp = """                
present(1, 1).
present(1, 2).
present(2,1).
timestep(1..2).
present(X, T):- visible(X, T).
initiate(interact(X), 5, 6):-present(X,T).
explore(X,Y):-present(X,T),present(Y,T).
rotate(T):-timestep(T).
interact(X):-initiate(interact(X)).
"""
# lp = """    present(1, 1).
# present(1, 2).
# present(2,1).
#             present(X,T):- visible(X, _, T).
#             object(X):-present(X,T).
#             occlusion(O):-visible(_,O,_).
#             explore(X,Y,O):-object(X), object(Y), occlusion(O).
#             interact(X):-object(X).
#             rotate."""
x1 = ASP(lp)
print(list(x1.parse_args))
x2 = ASP(lp)
# answer_sets = list(list(answer_sets)[0])
macro_actions = {
    "explore":3,
    "interact":1,
    "rotate":0
}
# filter_as = lambda x: [i for i in x if i[0] in macro_actions]
# filtered_as = filter_as(answer_sets)
# import itertools
from collections import deque, namedtuple

# print(list(x1.atoms_as_string.sorted))
# print(list(x2.parse_args.sorted))

In [ ]:
import subprocess

bashCommand = "ilasp --version=2i ../logic/logic4.lp -q"
process = subprocess.Popen(bashCommand, stdout=subprocess.PIPE, shell=True)
# # output, error = process.communicate()
# # output
# process = subprocess.Popen([ bashCommand.split(), stdout=subprocess.PIPE], shell=True)
output, error = process.communicate()
# subprocess.check_call(bashCommand, shell=True)
bool(output)

In [2]:
from collections import deque, namedtuple
import numpy as np
from clyngor import ASP
import subprocess

AnswerSet = namedtuple('AS', ['r', 'p', 'a']) # r for raw, p for parsed, a for arity


class Logic:
    def __init__(self, macro_step, global_step):
        self.macro_step = macro_step
        self.global_step = global_step
        self.grounder = Grounder()
        self.ilasp = Ilasp()
        self.clingo = Clingo()
        self.macro_actions = {
            "explore":3,
            "interact":1,
            "rotate":0
        }
        self.observables = [
            'present',
            'visible',
            'on',
            'adjacent',
            'goal',
            'goal1',
            'wall',
            'platform'
        ]
        self.e = 1
        self.e_discount = 1e-3

    def macro_kb(self):
        """This is what we want to learn."""
        return """
            0{initiate(explore(X,Y,Z),T)}1:- visible(X,Z,T), occludes(X,Y,T).
            initiate(interact(X),T):- visible(X, _,T), goal(X).
            initiate(rotate,T):- not visible(T), timestep(T).
        """
    def main_lp(self):
        return """
            % Observables rules
            present(X,T):- visible(X, _, T).
            visible(T):- visible(X, _, T).
            not_occluding(X, T):-on(agent, X, T).
            separator(Y, T):-on(agent, X, T), adjacent(X, Y, T), platform(X).
            occludes(X,Y,T) :- present(Y, T), visible(X, _, T), not visible(Y, _, T), not separator(X, T), not not_occluding(X, T).

            % Observables - > actions: this is what we need to learn
            :- initiate(explore(X1,Y,_), T), initiate(explore(X2,Y,_), T), X1 != X2.
            :~initiate(explore(X,Y,Z),T).[Z@1,X,Z]
            
            % Completion checks checks
            check(visible(Y),T):- initiate(explore(X,Y,Z),T).
            check(time, T):- initiate(explore(X,Y,Z),T).
            check(time, T):- initiate(interact(X),T).
            check(time, T):- initiate(rotate,T).
            """
    def e_greedy(self):
        # Don't start egreedy until there's at least one positive example with inclusion
        res = np.random.choice(['ilasp', 'random'], 1, p=[1-self.e, self.e])
        self.e = max(0.05, self.e - self.e_discount)
        return res

    def run(self, macro_step, state):
        # Need to have a list of all possible grounded macro actions
        # e greedy choice between ilasp grounded AS or random section from macro action list
        # If random choose random
        # If ilasp learns rules, add to lp and run clyngor. If returns initiate, use that
        # Otherwise use random
        # Gamma discount examples to ilasp as weights
        # Macro action list needs to be simple and translate easily to both modes and grounded mas
        # Need to keep timestep too, to control extent of timestep
        
        # Ground state into high level observable predicates
        observables = self.grounder.run(state)
        # E greedy decide to do random or ilasp
        choice = self.e_greedy()
        if choice == 'ilasp':
            lp = observables + main_lp
            learned_lp = self.ilasp.run(lp)
            if learned_lp:
                action = self.clingo.run(learned_lp, random=False)
            else:
                action = self.clingo.run(observables, random=True)
            # Need to update ilasp examples with outcome
        elif choice == 'random':
            action = self.clingo.run(observables, random=True)
        else:
            raise Exception("Modality not recognised")
        
        if not action['initiate']:
            action = self.clingo.run(observables, random=True)
        return action

class Grounder:
    def __init__(self):
        pass
    @staticmethod
    def adjacent(state):
        adjacent = ""
        for bbox, obj_type, _, _id in state['obj']:
            for bbox1, obj_type1, _, _id1 in state['obj']:
                dist = get_distance(bbox, bbox1)
                if (_id1!=_id)&(dist<0.02):
                    adjacent += f"adjacent({_id},{_id1}).\n"
        return adjacent
    @staticmethod
    def on(state):
        on = ""
        bottom_rect = [0, 0.75, 1, 0.25]
        for bbox, _, _, _id in state['obj']:
            if get_overlap(bbox, bottom_rect)>0.5:
                on += f"on(agent,{_id}).\n"
        return on

    @staticmethod
    def visible(state):
        visible = ""
        for _, obj_type, _occ_area, _id in state['obj']:
            visible += f"visible({_id},{_occ_area}).\n{obj_type}({_id}).\n"

        return visible
    @staticmethod
    def goal_visible(state):
        try:
            gg_id = next(i[3] for i in state['obj'] if i[1] in ['goal', 'goal1'])
        except StopIteration:
            gg_id = 42
        return f"goal({gg_id}).\n"
    def run(self, state):
        res = ""
        for k,v in self.__dict__.items():
            if isinstance(v, staticmethod):
                res+= getattr(self, v)(state)
        return res
class Ilasp(Logic):
    def __init__(self, macro_step, global_step):
        super().__init__(macro_step, global_step)
        # Examples are [int:weight, string:example]
        self.memory_len = 100
        self.examples = deque(maxlen=self.memory_len)
    def create_modeh(self):
        res = ""
        for name, num_preds in self.macro_actions.items():
            if num_preds:
                variables = "("
                for i in range(num_preds):
                    variables+=f"var(V{i}),"
                variables = variables[:-1]
            else:
                variables = ""
            res += f"#modeh(1, initiate({name}{variables},var(T))).\n"
        return res
    
    def update_examples(self, answer_set, valence):
        observables = '.'.join([i for i in answer_set.r if any(j in i for j in self.observables)])
        actions = ','.join([i for i in answer_set.r if any(j in i for j in self.macro_actons)])
        if valence=='success':
            example = f"#pos(a{self.memory_len}@{self.memory_len},{{actions}}, {{}}, {{observables}})"
        else:
            example = f"#pos(a{self.memory_len}@{self.memory_len},{{}}, {{actions}}, {{observables}})"
        self.examples.append([self.memory_len, example])
        for c, eg in enumerate(self.examples):
            at_index = eg[1].index[','] # first comma
            updated_eg = eg[1][:6] + f"{c}@{c}" + eg[1][at_index+1:]
            self.examples = [c, updated_eg]

    def run(self, lp):        
        # Create text file with lp
        with open("tmp.lp", "w") as text_file:
            text_file.write(lp+self.create_modeh())
        # Start bash process that runs ilasp learning
        bashCommand = "ilasp --version=2i tmp.lp -q"
        process = subprocess.Popen(bashCommand, stdout=subprocess.PIPE, shell=True)
        output, error = process.communicate()
        
        # Return new lp with learned rules
        if bool(output): #learned rules
            output = output.decode("utf-8")
            lp+= '\n' + output
            return lp
        return False # No learned rules, will choose random macro
        
        
class Clingo(Logic):
    def __init__(self, macro_step, global_step):
        super().__init__(macro_step, global_step)

    @staticmethod
    def random_action_grounder(ground_observables):
        lp = f"""
            {ground_observables}
            present(X,T):- visible(X, _, T).
            object(X):-present(X,T).
            occlusion(O):-visible(_,O,_).
            explore(X,Y,O):-object(X), object(Y), occlusion(O).
            interact(X):-object(X).
            rotate."""
        res = self.asp(lp)
        return random.choice([i for i in res if i in self.macro_actions])

    @staticmethod
    def asp(lp):
        as1 = list(ASP(lp).atoms_as_string.sorted)
        as2 = list(ASP(lp).parse_args.sorted)

        return AnswerSet(r=as1, p=as2, a=len(as1))


    def macro_processing(self, answer_set):
        # Look for initiate
        res = {
            'initiate':[],
            'check':[],
            'raw':[]
        }
        for c1, ans_set in enumerate(answer_set.p):
            for c2, literal in enumerate(ans_set):
                if 'initiate' in literal[0]:
                    res['initiate'].append(literal[1])
                    res['raw'].append(answer_set.r[c1][c2])
                if 'check' in literal[0]:
                    res['check'].append(literal[1])
        return res

    def run(lp, random=False):
        # Just ground macro actions based on observables
        if random:
            res = self.random_action_grounder(lp)
        else: # Run full lp
            res = self.asp(lp)
                
        return self.macro_processing(lp)



In [3]:
from collections import deque
x = deque(maxlen=3)
x.append(4)

In [4]:
x.append(5)

In [5]:
wa = {'sha'}
f"{wa}"

"{'sha'}"

In [6]:
''.join(['ba', 'la'])

'bala'

In [7]:
f"{{}}"

'{}'

In [8]:
"#pos(a{self.memory_len}@{self.memory_len},ba,"[41:]

',ba,'

In [9]:
e = 0.9
res = np.random.choice(['ilasp', 'random'], p=[e, 1-e])
res

'ilasp'

In [10]:
class Grounder:
    def __init__(self):
        print(vars(Grounder))
    @staticmethod
    def adjacent(macro_step,state):
        adjacent = ""
        for bbox, _, _, _id in state['obj']:
            for bbox1, _, _, _id1 in state['obj']:
                dist = get_distance(bbox, bbox1)
                if (_id1!=_id)&(dist<0.02):
                    adjacent += f"adjacent({_id},{_id1}, {macro_step}).\n"
        return adjacent
    @staticmethod
    def on(macro_step,state):
        on = ""
        bottom_rect = [0, 0.75, 1, 0.25]
        for bbox, _, _, _id in state['obj']:
            if get_overlap(bbox, bottom_rect)>0.5:
                on += f"on(agent,{_id},{macro_step}).\n"
        return on

    @staticmethod
    def visible(macro_step,state):
        visible = ""
        print("NOW HERE")
        for _, obj_type, _occ_area, _id in state['obj']:
            print(obj_type)
            visible += f"visible({_id},{_occ_area},{macro_step}).\n{obj_type}({_id}).\n"
        print(visible)
        return visible
    @staticmethod
    def goal_visible(_,state):
        try:
            gg_id = next(i[3] for i in state['obj'] if i[1] in ['goal', 'goal1'])
        except StopIteration:
            gg_id = 42
        return f"goal({gg_id}).\n"
    def run(self, macro_step, state):
        print("HERE")
        res = ""
        print(state)
        print(self.__dict__)
        for _,v in self.__dict__.items():
            print(v)

            if isinstance(v, staticmethod):
                res+= getattr(self, v)(macro_step,state)
                print(res)
        return res

In [11]:
x = Grounder()
vars(x)

{'__module__': '__main__', '__init__': <function Grounder.__init__ at 0x112cec488>, 'adjacent': <staticmethod object at 0x11371ce80>, 'on': <staticmethod object at 0x11371cfd0>, 'visible': <staticmethod object at 0x1141172b0>, 'goal_visible': <staticmethod object at 0x114117320>, 'run': <function Grounder.run at 0x112cec730>, '__dict__': <attribute '__dict__' of 'Grounder' objects>, '__weakref__': <attribute '__weakref__' of 'Grounder' objects>, '__doc__': None}


{}

In [15]:
list(ASP('initiate(interact(0),0).').parse_args)

[frozenset({('initiate', (('interact', (0,)), 0))})]